Online Developer Community Survey Data

By Michael A. Madaio, for the United Nations University, Institute on Computing and Society
<br> August, 2018

## Overview

Online developer communities boast millions of users - over 29 million on GitHub and over 8 million on Stack Overflow, in 2018. Participation in these communities is becoming one of the primary ways software developers learn new programming languages, improve their skills, develop collaborative projects, and find new job opportunities. [[1]](David and Shapiro, 2008) [[2]](Ford et al., 2016) [[3]](Vasilescu et al., 2015)  

Developers on these sites may ask and answer coding questions to improve their skills (e.g. Stack Overflow), use those skills to contribute to open-source code (e.g. GitHub) and participate in coding challenges (e.g. HackerRank). These platforms are becoming increasingly important to hiring decisions, as recruiters look at GitHub contributions or reputation on Stack Overflow as indicators of developers' skill.

However, despite the promise for these online developer communities to support software developers in their professional development, there are indicators that there may be serious difference in women and men's participation in these communities - differences which may further exacerbate existing gender gaps in the ICT workforce writ large.

## Usage

In this notebook, we provide functions for cleaning country names from surveys to standardize them with the ISO code. Many data sources use idiosyncratic country names, which makes joining with the ISO codes to use other regional data difficult. This notebook provides a process for cleaning the source data's country names using a "fuzzy" string matching with the ISO country names using Levenshtein distance.  

If you're running this in a Jupyter interface, click on each code block and press ctrl+Enter to run the code in it.  

Before each code block are text instructions that explain what those lines of code do.

## 1. Setup

These lines hide the code blocks with a toggle button to display them.

These lines import the necessary package dependencies.

In [56]:
import pandas as pd
import numpy as np
import os
import csv
import matplotlib.pyplot as plt
import pandas.tools.plotting
import seaborn as sns
import matplotlib
%matplotlib inline

# Install "fuzzywuzzy" and "python-Levenshtein" using pip install
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

plt.style.use('seaborn')


import plotly.offline as offline
offline.init_notebook_mode(True)
import plotly.graph_objs as go
#plt.style.use('ggplot')
pd.options.display.max_rows =300
plt.rc('ytick',labelsize=12)
plt.rc('ytick',labelsize=12)
plt.rc('axes',labelsize=12)

### 1.1. Load data

Define your input .csv file name here:

In [57]:
# Define source file name here

source_file = "survey_results_public.csv"

Run this block to load the data.

Make sure the input csv file is saved to the data folder.  
Make sure the country/region mapping csv (found [here](https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv)) is saved to the data folder as well.

In [69]:
data_folder = os.path.abspath('.\\data')

# Load source file
values = pd.read_csv('{0}\{1}'.format(data_folder, source_file), na_values=['#NULL!', 'nan'], low_memory=False)

# Load Country-Region data
country_mapping = pd.read_csv('{0}\Country-Region-Mapping.csv'.format(data_folder),encoding='latin-1')

### 1.1. Preview data

Preview column headers and sample values.   
Uncomment the print statements by deleting the # in front of the word print to run this block and see a (lengthy) preview of the data.

In [59]:
## Print number of respondents
print("Total respondents:", len(values), "\n")

## Print example values for columns
#print(values.head())

## Print list of column names
#print(values.columns.ravel())

## Print all columns
#values[0:3].transpose()

Total respondents: 98855 



### 1.2. Clean country names

Run this block to define the function created to match country names.

In [60]:
# Define function to check input names against ISO names

names_array=[]
ratio_array=[]

def match_names(input_names,correct_names):
    for row in input_names:
        x=process.extractOne(row, correct_names)
        names_array.append(x[0])
        ratio_array.append(x[1])
    return names_array,ratio_array


Run this block first to define the name of your country variable in your source data

In [61]:
# Change this to be the name of your country name column:

country_column_name = "Country"

Run this block first to store the list of unique country names from your source data set.

In [62]:
# Create a list of unique country names from source dataset
source_names=values[country_column_name].dropna().unique()

# Create a list of country names from ISO dataset
correct_names=country_mapping['name'].values

Run this block to create a list of the name matches, with the ratio of their match (i.e. how "close" a match they are. Higher is a closer match).

In [63]:
name_match, ratio_match = match_names(source_names,correct_names)

# Check to make sure the list of unique names in the name match is the same as the list of unique countries in the source list. 
# That is, there should be at least one matched name from ISO for each country name in your source data.

if len(name_match) != len(source_names):
    print(len(name_match))
    print(len(source_names))
    print("Error: Different numbers of unique country names. Re-run code and inspect the .csv's if you continue to get this error.")
else:
    print("You have", len(name_match), "country names.")


You have 183 country names.


In [64]:
print(name_match, source_names)

['Kenya', 'United Kingdom of Great Britain and Northern Ireland', 'United States of America', 'South Africa', 'Nigeria', 'India', 'Spain', 'Croatia', 'Netherlands', 'Israel', 'Sweden', 'Chile', 'Australia', 'Greece', 'Poland', 'Belgium', 'Argentina', 'Germany', 'Russian Federation', 'Indonesia', 'Ireland', 'France', 'Ukraine', 'Denmark', 'Dominican Republic', 'China', 'Latvia', 'Algeria', 'Colombia', 'Japan', 'Finland', 'Romania', 'Brazil', 'Bulgaria', 'Iran (Islamic Republic of)', 'Nepal', 'Bangladesh', 'Pakistan', 'Canada', 'Jordan', 'Portugal', 'Italy', 'New Zealand', 'Turkey', 'Central African Republic', 'Viet Nam', 'Austria', 'Egypt', 'Malaysia', 'Mozambique', 'Thailand', 'Slovakia', 'Lebanon', 'Mexico', 'Hong Kong', 'Norway', 'Myanmar', 'Switzerland', "Korea (Democratic People's Republic of)", 'Hungary', 'Armenia', 'Kazakhstan', 'United Arab Emirates', 'Singapore', 'Qatar', 'Bosnia and Herzegovina', 'Bahrain', 'Sri Lanka', 'Cambodia', 'Serbia', 'Philippines', 'Peru', 'Botswana', 

Run this block to generate a .csv file with the source names (from your data), the ISO name it was matched with, and the % match.  
Open this file and sort by % match to fix individual errors in country names (e.g. erroneously matching South Korea with Democratic People's Republic of Korea).  

In [65]:
# Write an output .csv file with the source names, ISO names, and percent match between them

# Define name of output file:
output_file = "country_names_for_checking.csv"

with open('{0}\\{1}'.format(data_folder, output_file), "w") as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(['Source Names', 'ISO Names', '% Match'])
        for i in range(0, len(source_names)):
            writer.writerow([source_names[i],name_match[i],ratio_match[i]])

Read fixed country names and store those in your source data frame:

In [66]:
# Read that fixed .csv and show the comparison to original source names

fixed = pd.read_csv("{0}\\{1}".format(data_folder, output_file),header=None,encoding="ISO-8859-1")
      
d = {'ISO Country Name':fixed[1][1:],'Original Country Name':source_names}
cross_ref =pd.DataFrame(d)

cross_ref.head()

,ISO Country Name,Original Country Name
1,Kenya,Kenya
2,United Kingdom of Great Britain and Northern I...,United Kingdom
3,United States of America,United States
4,South Africa,South Africa
5,Nigeria,Nigeria


Replace survey dataset country names with correct country names. 

In [67]:
# Creates a new column with the value for each row

# Define name of new column
new_country_column_name = "Country_Name"

values = values.join(cross_ref.set_index('Original Country Name'), on=country_column_name)
values = values.rename(index=str, columns={"ISO Country Name": new_country_column_name})  

# Join ISO labels to source data by country name
values = values.join(country_mapping.set_index('name'), on=new_country_column_name)

# Print the number of items for each country in your data
print(values[new_country_column_name].value_counts())


United States of America                                20309
India                                                   13721
Germany                                                  6459
United Kingdom of Great Britain and Northern Ireland     6221
Canada                                                   3393
Russian Federation                                       2869
France                                                   2572
Brazil                                                   2505
Poland                                                   2122
Australia                                                2018
Netherlands                                              1841
Spain                                                    1769
Italy                                                    1535
Ukraine                                                  1279
Sweden                                                   1248
Pakistan                                                 1050
China   